# Análisis exploratorio de incidencia delictiva en México

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data/datos_delitos.csv', encoding='latin-1')

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
meses = {'Enero': 1, 'Febrero': 2, 'Marzo': 3, 'Abril': 4, 'Mayo': 5, 'Junio': 6,
         'Julio': 7, 'Agosto': 8, 'Septiembre': 9, 'Octubre': 10, 'Noviembre': 11, 'Diciembre': 12}

df['mes_num'] = df['Mes'].map(meses)

df['Fecha'] = pd.to_datetime(df['Año'].astype(str) + '-' + df['mes_num'].astype(str) + '-01')

print("Columna Fecha creada con éxito.")

mis_estados = ['Colima', 'Jalisco', 'Sinaloa']
delito_interes = 'Homicidio doloso'

df_filtro = df[(df['Entidad'].isin(mis_estados)) & (df['Tipo de delito'] == delito_interes)]

df_grafica = df_filtro.groupby(['Fecha', 'Entidad']).size().unstack()

df_grafica.plot(kind='line', figsize=(14, 7), linewidth=2)
plt.title('Evolución de Homicidios Dolosos (2015-2019)')
plt.ylabel('Número de delitos')
plt.grid(True, alpha=0.3)
plt.show()

#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

In [ ]:
print("--- RESPUESTAS EJERCICIO 2 ---")

r1 = df[(df['Entidad'] == 'Colima') & (df['Año'] == 2018) & (df['Tipo de delito'] == 'Homicidio doloso')].shape[0]
print(f"1. Homicidios dolosos en Colima (2018): {r1}")

r2 = df[(df['Año'] == 2019) & (df['Tipo de delito'] == 'Robo de vehículo automotor')].shape[0]
print(f"2. Robos de vehículo (2019): {r2}")

delitos_graves = ['Homicidio doloso', 'Feminicidio']
r3 = df[df['Tipo de delito'].isin(delitos_graves)].groupby('Año').size()
print("\n3. Homicidios + Feminicidios por año:")
print(r3)

df_fem = df[df['Tipo de delito'] == 'Feminicidio']
r4 = df_fem.groupby(['Mes', 'Municipio']).size().sort_values(ascending=False).head(1)
print("\n4. Mes y Municipio con más feminicidios:")
print(r4)

r5 = df_fem.groupby(['Año', 'Entidad']).size().sort_values(ascending=False).head(1)
print("\n5. Año y Estado con más feminicidios:")
print(r5)

#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
anios = sorted(df['Año'].unique())

fig, axes = plt.subplots(nrows=1, ncols=len(anios), figsize=(20, 5))

for i, anio in enumerate(anios):
    datos_anio = df[df['Año'] == anio]
    
    conteo = datos_anio['Tipo de delito'].value_counts().head(5)
    
    axes[i].pie(conteo, labels=conteo.index, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f"Año {anio}")

plt.tight_layout()
plt.show()

---
#### Calcula la tasa por 100,000 habitantes
##### Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

No tienes que descargar nada. Ya están los datos en la carpeta data

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
reemplazos = {
    'Coahuila de Zaragoza': 'Coahuila',
    'Michoacán de Ocampo': 'Michoacán',
    'Veracruz de Ignacio de la Llave': 'Veracruz',
    'México': 'Estado de México' 
}
pobs['entidad'] = pobs['entidad'].replace(reemplazos)

df_completo = df.merge(pobs, left_on='Entidad', right_on='entidad', how='left')

df_tasas = df_completo.groupby(['Año', 'Entidad', 'poblacion']).size().reset_index(name='total_delitos')

df_tasas['tasa'] = (df_tasas['total_delitos'] / df_tasas['poblacion']) * 100000

print("¡Cálculo terminado! Aquí están las primeras tasas:")
df_tasas.head()